In [4]:
import math

activities_to_classify = {
        'Get/replace items from refrigerator/cabinets/drawers': 0,
        'Peel a cucumber': 1,
        'Clear cutting board': 2,
        'Slice a cucumber': 3,
        'Peel a potato': 4,
        'Slice a potato': 5,
        'Slice bread': 6,
        'Spread almond butter on a bread slice': 7,
        'Spread jelly on a bread slice': 8,
        'Open/close a jar of almond butter': 9,
        'Pour water from a pitcher into a glass': 10,
        'Clean a plate with a sponge': 11,
        'Clean a plate with a towel': 12,
        'Clean a pan with a sponge': 13,
        'Clean a pan with a towel': 14,
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 15,
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 16,
        'Stack on table: 3 each large/small plates, bowls': 17,
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 18,
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 19,
}

activities_to_verbs = {
        'Get/replace items from refrigerator/cabinets/drawers': 'Get',
        'Peel a cucumber': 'Peel',
        'Clear cutting board': 'Clear',
        'Slice a cucumber': 'Slice',
        'Peel a potato': 'Peel',
        'Slice a potato': 'Slice',
        'Slice bread': 'Slice',
        'Spread almond butter on a bread slice': 'Spread',
        'Spread jelly on a bread slice': 'Spread',
        'Open/close a jar of almond butter': 'Open/Close',
        'Pour water from a pitcher into a glass': 'Pour',
        'Clean a plate with a sponge': 'Clean',
        'Clean a plate with a towel': 'Clean',
        'Clean a pan with a sponge': 'Clean',
        'Clean a pan with a towel': 'Clean',
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Get',
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Set',
        'Stack on table: 3 each large/small plates, bowls': 'Stack',
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load',
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load'
}

verbs = {
    'Get': 0,
    'Peel': 1,
    'Clear': 2,
    'Slice': 3,
    'Spread': 4,
    'Open/Close': 5,
    'Pour': 6,
    'Clean': 7,
    'Set': 8,
    'Stack': 9,
    'Load': 10,
}



def fill_missing_intervals(df):
    df = df.sort_values(by='start_frame')

    dfs = []

    for i in range(len(df) - 1):
        current_row = df.loc[i]
        next_row = df.loc[i + 1]

        missing_interval_start = current_row['stop_frame'] + 1
        missing_interval_end = next_row['start_frame'] - 1

        if missing_interval_start <= missing_interval_end:
            new_row = pd.DataFrame({
                'uid': 90,
                'participant_id': 'S04',
                'video_id': 'S04_01',
                'narration': 'None',
                'verb': 'None',
                'verb_class': 0,
                'start_frame': [missing_interval_start],
                'stop_frame': [missing_interval_end],
                'start_timestamp': [missing_interval_start / 22],
                'stop_timestamp': [missing_interval_end / 22],
            })
            dfs.append(new_row)

    if dfs:
        filled_df = pd.concat([df] + dfs, ignore_index=True)
    else:
        filled_df = df.copy()

    return filled_df

def split_rows(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = 148  

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/22
            new_row['stop_timestamp'] = new_stop/22
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe



def create_annotations_file(timestamp_file, annotations_file_test, annotations_file_train):
    import pandas as pd

    video_fps = 29.67

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['start'].iloc[0]
    timestamps['start_timestamp'] = timestamps['start'] - start_timestamp
    timestamps['stop_timestamp'] = timestamps['stop'] - start_timestamp
    timestamps['start_frame'] = (timestamps['start_timestamp'] * video_fps).astype(int)
    timestamps['stop_frame'] = (timestamps['stop_timestamp'] * video_fps).astype(int)
    
    data_frame_train = pd.read_pickle(annotations_file_train)
    data_frame_train = data_frame_train[data_frame_train['file'] == 'S04_1.pkl']
    data_frame_train = data_frame_train.drop(['labels'], axis=1)
    data_frame_train['type'] = 'train'

    data_frame_test = pd.read_pickle(annotations_file_test)
    data_frame_test = data_frame_test[data_frame_test['file'] == 'S04_1.pkl']
    data_frame_test = data_frame_test.drop(['labels'], axis=1)
    data_frame_test['type'] = 'test'
    
    
    join_data_frame = pd.concat([data_frame_train, data_frame_test], ignore_index=True)

    result = pd.merge(timestamps, join_data_frame, on='index', how='inner')

    result['uid'] = result['index']
    result['participant_id'] = 'S04'
    result['video_id'] = 'S04_01'
    result['verb'] = result['description_x'].map(activities_to_verbs)
    result['narration'] = result['description_x']
    result['verb_class'] = result['verb'].map(verbs)

    result = result[
        ['uid', 'participant_id', 'video_id', 'narration', 'start_timestamp', 'stop_timestamp', 'start_frame',
         'stop_frame', 'verb', 'verb_class', 'type']]
            
    result = result.sort_values(by='start_frame')
    
    result.to_csv("JOINED_NOT_SPLIT.csv", index=False)

    result = split_rows(result)
    result['uid'] = range(len(result))
    
    test = result[result['type'] == 'test']
    test.drop('type', axis=1, inplace=True)
    test.to_csv("S04_test.csv", index=False)
    test.to_pickle("action_net_annotations/S04_test.pkl")
    train = result[result['type'] == 'train']
    train.drop('type', axis=1, inplace=True)
    train.to_csv("S04_train.csv", index=False)
    train.to_pickle("action_net_annotations/S04_train.pkl")

In [5]:
create_annotations_file('S04_1.pkl', 'action-net/ActionNet_test.pkl', 'action-net/ActionNet_train.pkl')

/var/folders/5s/flhrbc552q7bkv4fx1yszz1c0000gn/T/ipykernel_34580/2429511311.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('type', axis=1, inplace=True)
/var/folders/5s/flhrbc552q7bkv4fx1yszz1c0000gn/T/ipykernel_34580/2429511311.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('type', axis=1, inplace=True)


In [43]:
import pandas as pd

df = pd.read_pickle('saved_features_action_net/features_action_net_5_dense_S04_test.pkl')

df

{'features': [{'uid': 192,
   'video_name': 'S04_01',
   'features_RGB': array([[5.0480008e-01, 5.6000996e-01, 4.3215361e-01, ..., 1.0574757e+00,
           2.3778833e-02, 6.9013578e-01],
          [4.4164896e-01, 3.2415488e-01, 7.7726436e-01, ..., 3.7147805e-01,
           2.5176506e-02, 1.4859629e-01],
          [1.9551432e-01, 2.6093999e-01, 3.6436906e-01, ..., 6.0956115e-01,
           8.3428197e-02, 6.8431735e-02],
          [1.9629432e-01, 3.2201448e-01, 2.2445087e-01, ..., 3.2195324e-01,
           1.0356720e-03, 3.9699809e-03],
          [2.9793051e-01, 3.5625854e-01, 3.0813906e-01, ..., 1.8465391e-01,
           0.0000000e+00, 3.5283831e-01]], dtype=float32)},
  {'uid': 193,
   'video_name': 'S04_01',
   'features_RGB': array([[0.2196718 , 0.2452784 , 0.37475887, ..., 0.71281   , 0.01021689,
           0.45158803],
          [0.12478492, 0.4896867 , 0.22991219, ..., 0.40134367, 0.        ,
           0.23063774],
          [0.27958417, 0.13955842, 0.14225091, ..., 0.35581344, 

In [52]:
import pandas as pd

df = pd.read_pickle('action-net/ActionNet_test.pkl')

df = df[df['file'] == 'S04_1.pkl']

df

,index,file,description,labels
34,34,S04_1.pkl,Open/close a jar of almond butter,Open/Close
23,23,S04_1.pkl,Clear cutting board,Clear
26,26,S04_1.pkl,Spread almond butter on a bread slice,Spread
56,56,S04_1.pkl,"Set table: 3 each large/small plates, bowls, m...",Set
45,45,S04_1.pkl,Clean a plate with a towel,Clean
52,52,S04_1.pkl,Clean a pan with a towel,Clean
36,36,S04_1.pkl,Pour water from a pitcher into a glass,Pour
47,47,S04_1.pkl,Clean a plate with a towel,Clean


In [53]:
import pandas as pd

df = pd.read_pickle('action-net/ActionNet_train.pkl')

df = df[df['file'] == 'S04_1.pkl']

df

,index,file,description,labels
10,10,S04_1.pkl,Get/replace items from refrigerator/cabinets/d...,Get/Put
6,6,S04_1.pkl,Slice a cucumber,Slice
17,17,S04_1.pkl,Slice a potato,Slice
44,44,S04_1.pkl,Clean a plate with a sponge,Clean
37,37,S04_1.pkl,Pour water from a pitcher into a glass,Pour
27,27,S04_1.pkl,Spread almond butter on a bread slice,Spread
49,49,S04_1.pkl,Clean a pan with a sponge,Clean
33,33,S04_1.pkl,Open/close a jar of almond butter,Open/Close
29,29,S04_1.pkl,Spread jelly on a bread slice,Spread
24,24,S04_1.pkl,Get/replace items from refrigerator/cabinets/d...,Get/Put


In [3]:
import pandas as pd

df = pd.read_pickle('action_net_annotations/S04_test.pkl')

df

,uid,participant_id,video_id,narration,start_timestamp,stop_timestamp,start_frame,stop_frame,verb,verb_class
192,192,S04,S04_01,Clear cutting board,2693.636364,2700.318182,59260,59407,Clear cutting board,2
193,193,S04,S04_01,Clear cutting board,2700.363636,2707.045455,59408,59555,Clear cutting board,2
194,194,S04,S04_01,Clear cutting board,2707.090909,2713.772727,59556,59703,Clear cutting board,2
195,195,S04,S04_01,Clear cutting board,2713.818182,2720.500000,59704,59851,Clear cutting board,2
196,196,S04,S04_01,Clear cutting board,2720.545455,2727.227273,59852,59999,Clear cutting board,2
197,197,S04,S04_01,Clear cutting board,2727.272727,2730.681818,60000,60075,Clear cutting board,2
216,216,S04,S04_01,Spread almond butter on a bread slice,2890.500000,2897.181818,63591,63738,Spread almond butter on a bread slice,7
217,217,S04,S04_01,Spread almond butter on a bread slice,2897.227273,2903.909091,63739,63886,Spread almond butter on a bread slice,7
218,218,S04,S04_01,Spread almond butter on a bread slice,2903.954545,2910.636364,63887,64034,Spread almond butter on a bread slice,7
219,219,S04,S04_01,Spread almond butter on a bread slice,2910.681818,2913.863636,64035,64105,Spread almond butter on a bread slice,7


In [2]:
import pandas as pd

df = pd.read_pickle('saved_features_action_net/features_action_net_5_dense_S04_test.pkl')

df

{'features': [{'uid': 192,
   'video_name': 'S04_01',
   'features_RGB': array([[5.0480008e-01, 5.6000996e-01, 4.3215361e-01, ..., 1.0574757e+00,
           2.3778833e-02, 6.9013578e-01],
          [4.4164896e-01, 3.2415488e-01, 7.7726436e-01, ..., 3.7147805e-01,
           2.5176506e-02, 1.4859629e-01],
          [1.9551432e-01, 2.6093999e-01, 3.6436906e-01, ..., 6.0956115e-01,
           8.3428197e-02, 6.8431735e-02],
          [1.9629432e-01, 3.2201448e-01, 2.2445087e-01, ..., 3.2195324e-01,
           1.0356720e-03, 3.9699809e-03],
          [2.9793051e-01, 3.5625854e-01, 3.0813906e-01, ..., 1.8465391e-01,
           0.0000000e+00, 3.5283831e-01]], dtype=float32)},
  {'uid': 193,
   'video_name': 'S04_01',
   'features_RGB': array([[0.2196718 , 0.2452784 , 0.37475887, ..., 0.71281   , 0.01021689,
           0.45158803],
          [0.12478492, 0.4896867 , 0.22991219, ..., 0.40134367, 0.        ,
           0.23063774],
          [0.27958417, 0.13955842, 0.14225091, ..., 0.35581344, 